In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
#import xgboost as xgb
import lightgbm as lgb


In [2]:
from imp import reload
import matplotlib.pyplot as plt  
reload(plt)

<module 'matplotlib.pyplot' from 'D:\\ProgramData\\anaconda\\lib\\site-packages\\matplotlib\\pyplot.py'>

### 读取phishing data

In [3]:
phishing_data = pd.read_csv('./data/all_phishing_data.csv')

In [4]:
len(phishing_data)

70898

In [5]:
#phishing_data.groupby(['to'])['from'].agg(['count', 'nunique'])

In [6]:
#train_data.groupby(['to'])['from'].agg(['count', 'nunique']).mean()

In [296]:
#还未划分train test
frac = 0.001
def feature_matrix_to_train_matrix(features, train_label):
    train = pd.merge(left=features, right=train_label, left_on='address', right_on='address', how='inner')
    black_sample = train[train['label'] == 1]
    white_sample = train[train['label'] == 0].sample(n = 5 * len(black_sample))
    train = pd.concat([black_sample, white_sample])
    
    #打乱黑白样本
    train = train.sample(frac=1)
    #data = pd.merge(left=features, right=train_label, left_on='from', right_on='address', how='inner')
    #train = data[:int(len(data) * frac)]
    #test = data[int(len(data) * frac):]
    X_train = train.drop(['address', 'label'], axis=1)
    #X_train = X_train.sample(frac=frac, axis=1)
    y_train = train['label']
    #X_test = test.drop(['from', 'to', 'address', 'label'], axis=1)
    #y_test = test_label['label']
    return X_train, y_train

In [8]:
#y_train.sum() / len(y_train)

In [9]:
def wei2ether(x):
    x = list('0'.zfill(18) + x)
    x.insert(-18, '.')
    return np.double(''.join(x))

In [10]:
train_data = pd.read_csv('./data/train_data_ver2.0.csv')
train_label = pd.read_csv('./data/train_label_ver2.0.csv')

In [11]:
train_data['value'] = train_data['value'].apply(wei2ether)

In [12]:
train_label['label'].value_counts()

0    9099062
1       1220
Name: label, dtype: int64

In [13]:
len(train_data)

30956458

## 观察数据

In [14]:
len(set(train_data['to']))

8639415

In [15]:
def get_trival_feature(train_data):
    #feature between from and block:
    from_block_feature = train_data.groupby(['from'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('from_block_')
    from_value_feature = train_data.groupby(['from'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('from_value_')
    from_feature = pd.concat([from_block_feature, from_value_feature], axis = 1)
    #from_feature = from_feature.merge(left_on='from', right=train_address, right_on='address', how='right')
    to_block_feature = train_data.groupby(['to'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('to_block_')
    to_value_feature = train_data.groupby(['to'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('to_value_')
    to_feature = pd.concat([to_block_feature, to_value_feature], axis = 1)
    from_feature.reset_index(inplace=True)
    to_feature.reset_index(inplace=True)
    #to_feature = to_feature.merge(left_on='to', right=train_address, right_on='address', how='right')
    trival_feature = pd.merge(left=from_feature, right=to_feature, left_on='from', right_on='to', how='inner')
    trival_feature['address'] = trival_feature['from']
    trival_feature.drop(['from', 'to'], axis = 1, inplace=True)
    return trival_feature

In [16]:
train_address =train_label['address']

In [17]:
#pre sampled
train_data

,blockNumber,from,to,value
0,1917029,0xcf5cdd85813129271c907348325c013847674fc6,0x60aa80fe41b43c180870a137c7a87334e5c3cb5d,2.100000e-01
1,1917031,0x63656b599cb67a18b5be74f7dcd269137a6efdcb,0x7180eb39a6264938fdb3effd7341c4727c382153,9.995800e-01
2,1917031,0x63656b599cb67a18b5be74f7dcd269137a6efdcb,0x7180eb39a6264938fdb3effd7341c4727c382153,9.995800e-01
3,1917032,0xea321e7e27877e4e623a74b1262f184f404a8319,0x1e639f11abdd00fca7779aab75c3c3ed10c0d1a0,8.900000e-01
4,1917032,0x60aa80fe41b43c180870a137c7a87334e5c3cb5d,0xcf5cdd85813129271c907348325c013847674fc6,1.000000e-18
...,...,...,...,...
30956453,7006697,0xcdfaa6fd2461d7129a7fdc193c9d97504ff3b6f6,0x3b00a158410514fc4bbd40bf51f6fd0d140fb45d,1.800000e-01
30956454,7006697,0x20e6f8babbfdf132d049814698ec2281ae4b3da3,0xebe68414ab9a7d9124b685c9bb6f70d553a01664,1.789200e-03
30956455,7006699,0x29bc381c5a88d0fea8f60818725acceee9b981e7,0x968c6686b9d4f8f2aed5a6c5ea629f67cfc9ac2e,1.400000e+01
30956456,7006699,0xa2d3d545de4235044fff5116a4f8a043aaf13821,0x25bddd8c28d657f190d8e32cda1e29207718a7da,2.910000e+00


In [18]:
#feature between from and block:
from_block_feature = train_data.groupby(['from'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('from_block_')

In [19]:
from_value_feature = train_data.groupby(['from'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('from_value_')

In [20]:
from_feature = pd.concat([from_block_feature, from_value_feature], axis = 1)
#from_feature = from_feature.merge(left_on='from', right=train_address, right_on='address', how='right')
#from_feature.fillna(0, inplace=True)

In [21]:
to_block_feature = train_data.groupby(['to'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('to_block_')
to_value_feature = train_data.groupby(['to'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('to_value_')

In [22]:
to_feature = pd.concat([to_block_feature, to_value_feature], axis = 1)
#to_feature = to_feature.merge(left_on='to', right=train_address, right_on='address', how='right')

In [23]:
from_feature.reset_index(inplace=True)
to_feature.reset_index(inplace=True)

In [24]:
#merge方式

trival_feature = pd.merge(left=from_feature, right=to_feature, left_on='from', right_on='to', how='inner')

In [25]:
trival_feature

,from,from_block_ptp,from_block_std,from_value_sum,from_value_mean,from_value_std,from_value_max,from_value_min,to,to_block_ptp,to_block_std,to_value_sum,to_value_mean,to_value_std,to_value_max,to_value_min
0,0x0000000048246c5f37df029bef0cefd9ee36f301,35020,16237.896054,1.906650e-01,4.766626e-02,0.055247,1.282759e-01,9.346600e-03,0x0000000048246c5f37df029bef0cefd9ee36f301,93379,38911.802283,0.079356,0.019839,0.008781,0.029139,0.010000
1,0x000000004fdd41501a42651743c55f7e41ec85e4,322138,144046.283394,4.281103e-02,1.070276e-02,0.012553,2.620300e-02,1.000000e-05,0x000000004fdd41501a42651743c55f7e41ec85e4,0,NaN,0.054110,0.054110,NaN,0.054110,0.054110
2,0x00000000666bc8d3212838a58e92c18e6cfe9ef1,0,NaN,8.888889e-11,8.888889e-11,NaN,8.888889e-11,8.888889e-11,0x00000000666bc8d3212838a58e92c18e6cfe9ef1,1168154,826009.614870,0.070910,0.035455,0.050127,0.070900,0.000010
3,0x00000000ad4eb06bc550dc45ce28888c0626839a,80,22.360860,7.324813e+01,1.331784e+00,1.810276,9.183630e+00,1.006096e-01,0x00000000ad4eb06bc550dc45ce28888c0626839a,0,NaN,103.500000,103.500000,NaN,103.500000,103.500000
4,0x00000000b9d54fb8ab1a28ebd2ccbfe3247e517f,0,NaN,2.669227e+00,2.669227e+00,NaN,2.669227e+00,2.669227e+00,0x00000000b9d54fb8ab1a28ebd2ccbfe3247e517f,270001,145332.203372,43.790000,14.596667,6.829863,19.900000,6.890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7587228,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,2636291,418260.353952,5.332489e+03,2.866930e+01,144.015450,1.597000e+03,1.000000e-02,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,2055331,525883.200147,3136.556590,13.696754,35.252536,380.000000,0.001000
7587229,0xfffffe16439027b253808b7a55c787b67c22a75d,15,8.386497,1.009981e+02,3.366604e+01,48.429568,8.922260e+01,3.629000e-01,0xfffffe16439027b253808b7a55c787b67c22a75d,0,0.000000,101.000000,50.500000,54.158249,88.795665,12.204335
7587230,0xffffffa5473301b0489d7e1d1e7786b4b79e2cc0,117025,40840.724921,1.328789e+00,1.476433e-01,0.211307,5.500000e-01,3.000000e-03,0xffffffa5473301b0489d7e1d1e7786b4b79e2cc0,110058,39841.898372,0.165020,0.023574,0.015793,0.050000,0.004469
7587231,0xfffffff1ac8dcf49f4afd3f90995120e568740f9,696510,189335.232954,4.959864e+02,3.815280e+01,90.528909,2.500000e+02,9.864391e-01,0xfffffff1ac8dcf49f4afd3f90995120e568740f9,527774,28389.016979,1524.310398,3.928635,13.851119,200.000000,0.001000


In [26]:
trival_feature['address'] = trival_feature['from']
trival_feature.drop(['from', 'to'], axis = 1, inplace=True)

## 构建特征

### 网络特征——from, to

In [27]:
def get_network_feature(train_data):
    from_to_one_degree_feature = train_data.groupby(['from'])['to'].agg(['count', 'nunique']).add_prefix('from_to_').reset_index()
    from_to_one_degree_feature['from_to_count_nunique_ratio'] = from_to_one_degree_feature['from_to_count'] / from_to_one_degree_feature['from_to_nunique']
    from_to_one_degree_feature['from_to_count_nunique_equal'] = (from_to_one_degree_feature['from_to_count'] == from_to_one_degree_feature['from_to_nunique']).astype(int)
    from_to_two_degree_feature = train_data.groupby(['from', 'to'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['from']).agg(
        ['sum', 'std', 'median', 'max', 'min']).add_prefix('from_to_')
    from_to_two_degree_feature = pd.DataFrame(from_to_two_degree_feature.to_records(), index=None)
    from_to_degree_feature =  from_to_two_degree_feature.merge(right=from_to_one_degree_feature, left_on='from', right_on='from', how='inner')
                                                
        #to,from 特征以相同方式提取
    to_from_one_degree_feature = train_data.groupby(['to'])['from'].agg(['count', 'nunique']).add_prefix('to_from_').reset_index()
    to_from_one_degree_feature['to_from_count_nunique_ratio'] = to_from_one_degree_feature['to_from_count'] / to_from_one_degree_feature['to_from_nunique']
    to_from_one_degree_feature['to_from_count_nunique_equal'] = (to_from_one_degree_feature['to_from_count'] == to_from_one_degree_feature['to_from_nunique']).astype(int)
    
    to_from_two_degree_feature = train_data.groupby(['to', 'from'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['to']).agg(
    ['sum', 'std', 'median', 'max', 'min']).add_prefix('to_from_')
    to_from_two_degree_feature = pd.DataFrame(to_from_two_degree_feature.to_records(), index=None)
    to_from_degree_feature =  to_from_two_degree_feature.merge(right=to_from_one_degree_feature, left_on='to', right_on='to', how='inner')
    return from_to_degree_feature, to_from_degree_feature

In [28]:
#base: 每个from 节点对应的 to 节点的（不同）个数 (one_degree)
from_to_one_degree_feature = train_data.groupby(['from'])['to'].agg(['count', 'nunique']).add_prefix('from_to_').reset_index()
from_to_one_degree_feature['from_to_count_nunique_ratio'] = from_to_one_degree_feature['from_to_count'] / from_to_one_degree_feature['from_to_nunique']
from_to_one_degree_feature['from_to_count_nunique_equal'] = (from_to_one_degree_feature['from_to_count'] == from_to_one_degree_feature['from_to_nunique']).astype(int)
# 还可做： count/ nunique, count == nunique?

In [29]:
#统计from 节点相连接的 to 节点的信息情况

In [30]:
from_to_two_degree_feature = train_data.groupby(['from', 'to'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['from']).agg(
    ['sum', 'std', 'median', 'max', 'min']).add_prefix('from_to_')
from_to_two_degree_feature = pd.DataFrame(from_to_two_degree_feature.to_records(), index=None)

In [31]:
from_to_degree_feature =  from_to_two_degree_feature.merge(right=from_to_one_degree_feature, left_on='from', right_on='from', how='inner')

In [32]:
#to,from 特征以相同方式提取
to_from_one_degree_feature = train_data.groupby(['to'])['from'].agg(['count', 'nunique']).add_prefix('to_from_').reset_index()
to_from_one_degree_feature['to_from_count_nunique_ratio'] = to_from_one_degree_feature['to_from_count'] / to_from_one_degree_feature['to_from_nunique']
to_from_one_degree_feature['to_from_count_nunique_equal'] = (to_from_one_degree_feature['to_from_count'] == to_from_one_degree_feature['to_from_nunique']).astype(int)

In [33]:
to_from_two_degree_feature = train_data.groupby(['to', 'from'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['to']).agg(
    ['sum', 'std', 'median', 'max', 'min']).add_prefix('to_from_')
to_from_two_degree_feature = pd.DataFrame(to_from_two_degree_feature.to_records(), index=None)

In [34]:
to_from_degree_feature =  to_from_two_degree_feature.merge(right=to_from_one_degree_feature, left_on='to', right_on='to', how='inner')

### 特征拼接

In [35]:
def get_feature(train_data):
    trival_feature = get_trival_feature(train_data)
    from_to_degree_feature, to_from_degree_feature = get_network_feature(train_data)
    feature = trival_feature.merge(right=from_to_degree_feature, left_on='address', right_on='from', how='left')
    feature = feature.merge(right=to_from_degree_feature, left_on='address', right_on='to',how='left')
    feature.drop(['from','to'], inplace=True, axis = 1)
    return feature

In [36]:
feature = trival_feature.merge(right=from_to_degree_feature, left_on='address', right_on='from', how='left')

In [37]:
feature = feature.merge(right=to_from_degree_feature, left_on='address', right_on='to',how='left')

In [38]:
feature.drop(['from','to'], inplace=True, axis = 1)

In [39]:
def deal_feature_name(x):
    if x.count('from_to_') == 2:
        return 'from_in_' + x[1:-1].replace('from_to_', '').replace(',', '').replace('\'', '').replace(' ', '_')
    if x.count('to_from_') == 2:
        return 'to_out_' + x[1:-1].replace('to_from_', '').replace(',', '').replace('\'', '').replace(' ', '_')
    if x.count('from_to_') == 1:
        return x.replace('from_to', 'out')
    if x.count('to_from_') == 1:
        return x.replace('to_from', 'in')
    if (x.count('block') == 1 or x.count('value') == 1):
        return x.replace('from_block', 'out_block').replace('from_value', 'out_value').replace('to_block', 'in_block').replace('to_value', 'in_value')
    return x

In [40]:
feature = get_feature(train_data)
#处理特征名称
feature.columns = list(map(deal_feature_name,feature.columns))

In [233]:
feature.shape

(7587233, 63)

### 序列特征

In [214]:
#将数据集反过来重复一次，value取负数。以便于构建序列特征
reverse_train_data = train_data.copy(deep=True)
reverse_train_data.columns = ['blockNumber','to','from','value']
reverse_train_data['value']*= -1
double_dup_data = pd.concat([train_data, reverse_train_data],axis = 0)

In [224]:
#按时间排序
double_dup_data.sort_index(inplace=True)

In [225]:
#sample_data = train_data.sample(frac=0.001)
seq_len = 10
seq_diff0_to_val_feature = train_data.groupby(['to'])['value'].apply(lambda x: pd.Series(list(x[-seq_len:]) + (seq_len - len(x[-seq_len:])) * [np.nan])).unstack().add_prefix('diff0_to_val_').reset_index()
seq_diff0_from_val_feature = train_data.groupby(['from'])['value'].apply(lambda x: pd.Series(list(x[-seq_len:]) + (seq_len - len(x[-seq_len:])) * [np.nan])).unstack().add_prefix('diff0_from_val_').reset_index()
seq_diff0_to_blocknum_feature = train_data.groupby(['to'])['blockNumber'].apply(lambda x: pd.Series(list(x[-seq_len:]) + (seq_len - len(x[-seq_len:])) * [np.nan])).unstack().add_prefix('diff0_to_blocknum_').reset_index()
seq_diff0_from_blocknum_feature = train_data.groupby(['from'])['blockNumber'].apply(lambda x: pd.Series(list(x[-seq_len:]) + (seq_len - len(x[-seq_len:])) * [np.nan])).unstack().add_prefix('diff0_from_blocknum_').reset_index()


In [226]:
seq_diff0_val_feature = double_dup_data.groupby(['to'])['value'].apply(lambda x: pd.Series(list(x[-seq_len:]) + (seq_len - len(x[-seq_len:])) * [np.nan])).unstack().add_prefix('diff0_val_').reset_index()
seq_diff0_blocknum_feature = double_dup_data.groupby(['to'])['blockNumber'].apply(lambda x: pd.Series(list(x[-seq_len:]) + (seq_len - len(x[-seq_len:])) * [np.nan])).unstack().add_prefix('diff0_blocknum_').reset_index()


In [231]:
seq_diff0_to_val_feature.sort_values(by='to', inplace=True)
seq_diff0_from_val_feature.sort_values(by='from', inplace=True)
seq_diff0_to_blocknum_feature.sort_values(by='to', inplace=True)
seq_diff0_from_blocknum_feature.sort_values(by='from', inplace=True)
seq_diff0_val_feature.sort_values(by='to', inplace=True)
seq_diff0_blocknum_feature.sort_values(by='to', inplace=True)

seq_diff0_to_feature = pd.concat([seq_diff0_to_val_feature,
                                  seq_diff0_to_blocknum_feature.drop(['to'], axis = 1),
                                  seq_diff0_val_feature.drop(['to'], axis = 1),
                                  seq_diff0_blocknum_feature.drop(['to'], axis = 1)], axis = 1)

seq_diff0_from_feature = pd.concat([seq_diff0_from_val_feature, seq_diff0_from_blocknum_feature.drop(['from'], axis = 1)], axis = 1)

In [237]:
seq_diff0_feature = pd.merge(left=seq_diff0_to_feature, right=seq_diff0_from_feature, left_on='to', right_on='from', how='inner')

In [243]:
seq_diff0_feature.drop(['from'],axis = 1, inplace=True)

In [253]:
#bug-fixed code
#seq_diff0_feature.columns = list(map(lambda x: x.replace('diff0_from_val', 'diff0_from_val_') if ('diff0_from_val' in x)and ('diff0_from_val_' not in x) else x, seq_diff0_feature.columns))

In [255]:
#seq_feature = seq_diff0_feature.copy(deep=True)
#这里控制需要循环得到的序列特征层数（平稳化阶数）,一般最多到2阶
arima_D = 2
seq_fea_list = [seq_diff0_feature.copy(deep=True)]
for k in range(1, arima_D + 1):
    seq_integrate_fea = pd.DataFrame()
    seq_integrate_fea['to'] = seq_diff0_feature['to']
    for i in range(k, seq_len):
        seq_integrate_fea['diff' + str(k) + '_to_val_' + str(i)] = seq_fea_list[k - 1]['diff' + str(k - 1) + '_to_val_' + str(i)] - seq_fea_list[k - 1]['diff' + str(k - 1) + '_to_val_' + str(i - 1)]
        seq_integrate_fea['diff' + str(k) + '_from_val_' + str(i)] = seq_fea_list[k - 1]['diff' + str(k - 1) + '_from_val_' + str(i)] - seq_fea_list[k - 1]['diff' + str(k - 1) + '_from_val_' + str(i - 1)]
        seq_integrate_fea['diff' + str(k) + '_to_blocknum_' + str(i)] = seq_fea_list[k - 1]['diff' + str(k - 1) + '_to_blocknum_' + str(i)] - seq_fea_list[k - 1]['diff' + str(k - 1) + '_to_blocknum_' + str(i - 1)]
        seq_integrate_fea['diff' + str(k) + '_from_blocknum_' + str(i)] = seq_fea_list[k - 1]['diff' + str(k - 1) + '_from_blocknum_' + str(i)] - seq_fea_list[k - 1]['diff' + str(k - 1) + '_from_blocknum_' + str(i - 1)]
        seq_integrate_fea['diff' + str(k) + '_val_' + str(i)] = seq_fea_list[k - 1]['diff' + str(k - 1) + '_val_' + str(i)] - seq_fea_list[k - 1]['diff' + str(k - 1) + '_val_' + str(i - 1)]
        seq_integrate_fea['diff' + str(k) + '_blocknum_' + str(i)] = seq_fea_list[k - 1]['diff' + str(k - 1) + '_blocknum_' + str(i)] - seq_fea_list[k - 1]['diff' + str(k - 1) + '_blocknum_' + str(i - 1)]
    seq_fea_list.append(seq_integrate_fea.copy(deep=True))

In [256]:
seq_feature = pd.concat(seq_fea_list, axis = 1)
seq_feature['address'] = seq_diff0_feature['to']
seq_feature.drop(['to'], axis = 1, inplace=True)

## 单独测试序列特征

In [281]:
seq_X_train, seq_y_train = feature_matrix_to_train_matrix(seq_feature, train_label)

In [282]:
seq_dtrain = lgb.Dataset(seq_X_train, label = seq_y_train)

In [287]:
cv_res = lgb.cv(lgb_params, seq_dtrain, num_boost_round=800,feval=[lgb_evalMetric, lgb_pre_evalMetric,lgb_recall_evalMetric], verbose_eval = 10, nfold=3,metrics=('lgb_evalMetric'),
       early_stopping_rounds= 800)

[10]	cv_agg's res: 0.9277 + 0.0645453
[20]	cv_agg's res: 0.93172 + 0.0489559
[30]	cv_agg's res: 0.931981 + 0.0449017
[40]	cv_agg's res: 0.935988 + 0.0403676
[50]	cv_agg's res: 0.935307 + 0.0403963
[60]	cv_agg's res: 0.936454 + 0.0395567
[70]	cv_agg's res: 0.936976 + 0.0380832
[80]	cv_agg's res: 0.936694 + 0.0395093
[90]	cv_agg's res: 0.936075 + 0.0377049
[100]	cv_agg's res: 0.936034 + 0.0377248
[110]	cv_agg's res: 0.936796 + 0.0373867
[120]	cv_agg's res: 0.937436 + 0.0361427
[130]	cv_agg's res: 0.93705 + 0.0363328
[140]	cv_agg's res: 0.937769 + 0.0356011
[150]	cv_agg's res: 0.938084 + 0.0350811
[160]	cv_agg's res: 0.938052 + 0.0350411
[170]	cv_agg's res: 0.937761 + 0.0356065
[180]	cv_agg's res: 0.938077 + 0.0350647
[190]	cv_agg's res: 0.938559 + 0.035277
[200]	cv_agg's res: 0.938061 + 0.0350523
[210]	cv_agg's res: 0.938037 + 0.0350215
[220]	cv_agg's res: 0.938001 + 0.0349911
[230]	cv_agg's res: 0.938454 + 0.0348078
[240]	cv_agg's res: 0.938447 + 0.0347899
[250]	cv_agg's res: 0.938448 +

In [262]:
max(cv_res['res-mean'])

0.9518251213175244

## 按时间做OOT切割

In [297]:
X_train, y_train = feature_matrix_to_train_matrix(feature, train_label)

In [298]:
X_train.sort_index(inplace=True)
y_train.sort_index(inplace=True)

In [299]:
split_frac = 1 / 3
split_point = int((1 - split_frac) * len(X_train))
oot_train = X_train[0:split_point]
oot_train_y = y_train[0:split_point]
oot_test = X_train[split_point:]
oot_test_y = y_train[split_point:]

In [300]:
oot_test

,out_block_ptp,out_block_std,out_value_sum,out_value_mean,out_value_std,out_value_max,out_value_min,in_block_ptp,in_block_std,in_value_sum,...,to_out_max_min,to_out_min_sum,to_out_min_std,to_out_min_median,to_out_min_max,to_out_min_min,in_count,in_nunique,in_count_nunique_ratio,in_count_nunique_equal
4994426,1,0.707107,100.297871,50.148935,43.022489,80.570429,19.727442,4,2.828427,101.0,...,9.548549,101.0,57.914098,50.500000,91.451451,9.548549,2,2,1.0,1
4999520,19,9.712535,48.912640,16.304213,27.436127,47.983890,0.265380,1,0.707107,101.0,...,14.259854,101.0,51.251306,50.500000,86.740146,14.259854,2,2,1.0,1
5000117,635530,341024.212165,4.430000,1.107500,1.875658,3.900000,0.010000,0,NaN,0.1,...,0.100000,0.1,NaN,0.100000,0.100000,0.100000,1,1,1.0,1
5000593,820899,299997.970203,24.473200,3.496171,3.303878,10.000000,0.001000,0,NaN,10.0,...,10.000000,10.0,NaN,10.000000,10.000000,10.000000,1,1,1.0,1
5002056,5,2.886751,100.998740,33.666247,38.631396,76.710894,2.007184,1,0.707107,101.0,...,26.445508,101.0,34.018188,50.500000,74.554492,26.445508,2,2,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7578545,0,0.000000,100.581220,50.290610,37.380673,76.722737,23.858483,0,0.000000,101.0,...,32.934178,101.0,24.841824,50.500000,68.065822,32.934178,2,2,1.0,1
7579990,0,0.000000,100.798740,50.399370,35.257055,75.329873,25.468867,1,0.707107,101.0,...,40.028447,101.0,14.809012,50.500000,60.971553,40.028447,2,2,1.0,1
7582280,572466,229861.258283,28.900000,5.780000,6.883095,17.500000,0.100000,495590,199821.744521,14.3,...,0.500000,11.3,4.656447,0.650000,9.800000,0.200000,6,4,1.5,0
7586044,6,4.242641,199.816237,99.908119,75.650130,153.400838,46.415399,9,4.932883,201.0,...,13.504406,201.0,73.121337,37.175755,150.319839,13.504406,3,3,1.0,1


In [301]:
oot_dtrain = lgb.Dataset(oot_train, label = oot_train_y, free_raw_data=False)
oot_dtest = lgb.Dataset(oot_test, label = oot_test_y, free_raw_data=False)

In [303]:
evals_result = {}
#threshold=0.3
lgb_model = lgb.train(lgb_params, oot_dtrain, num_boost_round=1200,feval=[lgb_evalMetric, lgb_recall_evalMetric, lgb_pre_evalMetric], verbose_eval = 5,valid_sets=oot_dtest,evals_result=evals_result,#非常重要的参数,一定要明确设置
                       
                      early_stopping_rounds= 400)

Training until validation scores don't improve for 400 rounds
[5]	valid_0's res: 0.972047	valid_0's res: 0.820833	valid_0's res: 0.757692
[10]	valid_0's res: 0.97515	valid_0's res: 0.829167	valid_0's res: 0.745318
[15]	valid_0's res: 0.974926	valid_0's res: 0.8375	valid_0's res: 0.752809
[20]	valid_0's res: 0.975733	valid_0's res: 0.820833	valid_0's res: 0.746212
[25]	valid_0's res: 0.975847	valid_0's res: 0.825	valid_0's res: 0.744361
[30]	valid_0's res: 0.976488	valid_0's res: 0.825	valid_0's res: 0.755725
[35]	valid_0's res: 0.976332	valid_0's res: 0.825	valid_0's res: 0.758621
[40]	valid_0's res: 0.976589	valid_0's res: 0.825	valid_0's res: 0.752852
[45]	valid_0's res: 0.976595	valid_0's res: 0.825	valid_0's res: 0.761538
[50]	valid_0's res: 0.976703	valid_0's res: 0.825	valid_0's res: 0.767442
[55]	valid_0's res: 0.976819	valid_0's res: 0.816667	valid_0's res: 0.780876
[60]	valid_0's res: 0.976399	valid_0's res: 0.804167	valid_0's res: 0.781377
[65]	valid_0's res: 0.976767	valid_0

In [ ]:
fpr,tpr,threshold = roc_curve(y_test, y_score) ###计算真正率和假正率
roc_auc = auc(fpr,tpr) ###计算auc的值
 
plt.figure()
lw = 2
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [43]:
len(y_train)

8385

## 模型测试

In [147]:
threshold = 0.5
def lgb_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    #pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    auc = metrics.roc_auc_score(y_true=pre.label,y_score=pre.preds)
    #precision = metrics.precision_score(pre.label,pre.preds)
    #recall = metrics.recall_score(pre.label,pre.preds)
    res = auc
    return 'res',res, True

def lgb_pre_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    precision = metrics.precision_score(pre.label,pre.preds)
    res = precision
    return 'res',res, True

def lgb_recall_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    recall = metrics.recall_score(pre.label,pre.preds)
    res = recall
    return 'res',res, True

def lgb_auc_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    res = auc
    return 'res',res, True

In [148]:
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    #'colsample_bytree': 0.8,
    'learning_rate': 0.2,
    'num_leaves': 32,
    'verbosity': -1,
    'max_depth': 10,
    'metric': 'lgb_recall_evalMetric'
    }

In [149]:
dtrain = lgb.Dataset(X_train, label = y_train)

In [150]:
cv_res = lgb.cv(lgb_params, dtrain, num_boost_round=250,feval=lgb_evalMetric, verbose_eval = 10, nfold=3,metrics=('lgb_evalMetric'),
       early_stopping_rounds= 100)

[10]	cv_agg's res: 0.930148 + 0.00788435
[20]	cv_agg's res: 0.935339 + 0.00377644
[30]	cv_agg's res: 0.946376 + 0.00380912
[40]	cv_agg's res: 0.948239 + 0.00307759
[50]	cv_agg's res: 0.950972 + 0.00259789
[60]	cv_agg's res: 0.953408 + 0.00171131
[70]	cv_agg's res: 0.955834 + 0.00183767
[80]	cv_agg's res: 0.958261 + 0.00120046
[90]	cv_agg's res: 0.95988 + 0.00163946
[100]	cv_agg's res: 0.961089 + 0.00121944
[110]	cv_agg's res: 0.96193 + 0.00165268
[120]	cv_agg's res: 0.962903 + 0.00295988
[130]	cv_agg's res: 0.963979 + 0.00309253
[140]	cv_agg's res: 0.96445 + 0.00319388
[150]	cv_agg's res: 0.965309 + 0.00353304
[160]	cv_agg's res: 0.96628 + 0.00285123
[170]	cv_agg's res: 0.966833 + 0.00277608
[180]	cv_agg's res: 0.966839 + 0.00322851
[190]	cv_agg's res: 0.966749 + 0.00283177
[200]	cv_agg's res: 0.966708 + 0.00265914
[210]	cv_agg's res: 0.966868 + 0.00270874
[220]	cv_agg's res: 0.967464 + 0.00263151
[230]	cv_agg's res: 0.967761 + 0.00316235
[240]	cv_agg's res: 0.968047 + 0.00273348
[250]

In [307]:
phishing_address_set = set(train_label[train_label['label'] == 1]['address'])

In [308]:
phishing_data = train_data[(train_data['from'] in phishing_address_set)|(train_data['to'] in phishing_address_set)]

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [ ]:
while True:    
    X_train, y_train = feature_matrix_to_train_matrix(feature, train_label)
    dtrain = lgb.Dataset(X_train, label = y_train)
    evals_result = {}
    lgb_model = lgb.train(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10,valid_sets=dtrain,evals_result=evals_result,#非常重要的参数,一定要明确设置
           early_stopping_rounds= 100)
    sorted_impo = pd.DataFrame([lgb_model.feature_name(), lgb_model.feature_importance()]).T.sort_values(by=1, ascending=False)[0]
    if (sorted_impo[1] == 'to_out_sum_median' and sorted_impo[2] == 'from_in_sum_mean') :break

In [ ]:
lgb.plot_importance(lgb_model, max_num_features=15, title='')
plt.savefig('importance.jpg')

In [ ]:
sorted_impo = pd.DataFrame([lgb_model.feature_name(), lgb_model.feature_importance()]).T.sort_values(by=1, ascending=False)[0]

### 动态性测试

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#按時間劃分
cv_res = []
frac_num = 15
for i in range(frac_num):
    curr_train = train_data[0:int(len(train_data) * ((i + 1) / frac_num))]

    curr_feature = get_feature(curr_train)

    X_curr, y_curr = feature_matrix_to_train_matrix(curr_feature, train_label)

    dtrain = lgb.Dataset(X_curr, label = y_curr)

    cv_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_evalMetric'),
           early_stopping_rounds= 100)

    cv_res.append(max(cv_tmp['res-mean']))

In [ ]:
cv_res

In [ ]:
phishing_label = pd.read_csv('./data/phishing_label.csv')

In [ ]:
len(train_data)

In [ ]:
address_map = {}
def cal_data_index(x):
    if x in address_map:
        address_map[x] = address_map[x] + 1
    else:
        address_map[x] = 1
    return address_map[x]

In [ ]:
train_data['data_index'] = train_data['to'].apply(cal_data_index)

In [ ]:
upperbound = 59
cv_pre_res = []
cv_recall_res = []
cv_auc_res = []
for i in range(upperbound):
    kth_train_address_set = set(train_data[train_data['data_index'] == i + 1]['to'])
    kth_train_data = train_data[train_data['data_index'] <= i + 1]
    kth_train_data = kth_train_data[kth_train_data['to'].isin(kth_train_address_set)]
    kth_train_feature = get_feature(kth_train_data)
    X_kth, y_kth = feature_matrix_to_train_matrix(kth_train_feature, train_label)
    dtrain = lgb.Dataset(X_kth, label = y_kth)
    cv_pre_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_pre_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_pre_evalMetric'),
           early_stopping_rounds= 100)
    cv_recall_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_recall_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_recall_evalMetric'),
           early_stopping_rounds= 100)   
    cv_auc_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_auc_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_auc_evalMetric'),
           early_stopping_rounds= 100)      
    cv_pre_res.append(max(cv_pre_tmp['res-mean']))
    cv_recall_res.append(max(cv_recall_tmp['res-mean']))
    cv_auc_res.append(max(cv_auc_tmp['res-mean']))

In [ ]:
cv_f1_res = list(map(lambda x, y:  2 * x * y / (x + y), cv_pre_res, cv_recall_res))

In [ ]:
cv_pre_res[15]

In [ ]:
index = list(range(len(cv_auc_res)))

In [ ]:
import matplotlib.pyplot as plt
x = np.array(index)


y_pre = np.array(cv_pre_res)
f_fit = np.poly1d(np.polyfit(x, y_pre, 12))
yfit = f_fit(x)
plt.plot(x, yfit, label='Precision')

y_recall = np.array(cv_recall_res)
f_fit = np.poly1d(np.polyfit(x, y_recall, 7))
yfit = f_fit(x)
plt.plot(x, yfit, label='Recall')

y_f1 = np.array(cv_f1_res)
f_fit = np.poly1d(np.polyfit(x, y_f1, 10))
yfit = f_fit(x)
plt.plot(x, yfit, label='F1')

y_auc = np.array(cv_auc_res)
f_fit = np.poly1d(np.polyfit(x, y_auc, 10))
yfit = f_fit(x)
plt.plot(x, yfit, label='AUC')

plt.xlabel('Number of in transaction')
plt.ylabel('Cross validation performance')
plt.legend()
plt.savefig('dynamic_performance.jpg')

In [ ]:
train_label.sum()

In [ ]:
feature_new = get_feature(train_data)

In [ ]:
X_tmp, y_tmp = feature_matrix_to_train_matrix(feature, train_label)
dtrain = lgb.Dataset(X_tmp, label = y_tmp)
cv_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10, nfold=5,metrics=('lgb_evalMetric'),
           early_stopping_rounds= 100)

In [ ]:
y_tmp.sum()

### 與svm和dt對比

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_tn, x_tt, y_tn, y_tt = train_test_split(X_train,y_train, test_size=0.25)

In [ ]:
from sklearn import svm

In [ ]:
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
x_tn.fillna(0, inplace=True)
x_tt.fillna(0, inplace=True)
clf.fit(x_tn, y_tn)
svm_preds = clf.predict(x_tt)
svm_auc_score = roc_auc_score(y_tt,svm_preds)
svm_precision_score = precision_score(y_tt, svm_preds, average='macro')
svm_recall_score = recall_score(y_tt, svm_preds, average='macro')

In [ ]:
clf = tree.DecisionTreeClassifier()
x_tn.fillna(0, inplace=True)
x_tt.fillna(0, inplace=True)
clf.fit(x_tn, y_tn)
dt_preds = clf.predict(x_tt)
dt_auc_score = roc_auc_score(y_tt,dt_preds)
dt_precision_score = precision_score(y_tt, dt_preds, average='macro')
dt_recall_score = recall_score(y_tt, dt_preds, average='macro')

In [ ]:
#经过均衡
print(dt_auc_score)
print(dt_precision_score)
print(dt_recall_score)

## 案例分析

In [ ]:
sample_addr = '0x2a6d8021861f27ab992572d8689017b7a83c989d'

In [ ]:
sample_addr_data = train_data[(train_data['from'] == sample_addr) | (train_data['to'] == sample_addr)]

In [ ]:
sample_addr_data

In [ ]:
#sample_addr_data

In [ ]:
sample_dynamic_feature = pd.DataFrame()
for i in range(len(sample_addr_data)):
    part_sample_addr_data = sample_addr_data[:i]
    tmp_fea = get_sample_feature(part_sample_addr_data)
    target_fea = tmp_fea[tmp_fea['address'] == sample_addr]
    sample_dynamic_feature = pd.concat([sample_dynamic_feature,target_fea])

In [ ]:
sample_dynamic_feature.drop('address', axis = 1, inplace=True)

In [ ]:
without_sample_fea = feature[feature['address'] != sample_addr]
X_tmp, y_tmp = feature_matrix_to_train_matrix(without_sample_fea, train_label)
dtrain = lgb.Dataset(X_tmp, label = y_tmp)
sample_lgb_model = lgb.train(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10,valid_sets=dtrain,
       early_stopping_rounds= 100)

In [ ]:
sample_dynamic_res = sample_lgb_model.predict(sample_dynamic_feature)

In [ ]:
sample_dynamic_res

In [ ]:
dynamic_x = np.array(range(len(sample_dynamic_res)))
dynamic_f_fit = np.poly1d(np.polyfit(dynamic_x, sample_dynamic_res,5))
dynamic_yfit = dynamic_f_fit(dynamic_x)
plt.plot(dynamic_x, dynamic_yfit, marker='o', markevery=[3,15])
plt.axhline(y=0.52306926,ls='--', xmax=0.06, color='r')
plt.axvline(x=3, ls='--',ymax=0.25, color='r')
plt.annotate("(4, 0.523)", xy=(4, 0.52), xytext=(4, 2), textcoords='offset points')
plt.axhline(y=0.9078484 ,ls='--', xmax=0.15,color='r')
plt.axvline(x=15, ls='--',ymax=0.77,color='r')
plt.annotate("(16, 0.908)", xy=(15, 0.908), xytext=(6, 2), textcoords='offset points')
plt.xlabel('Number of transactions')
plt.ylabel('P(phishing address)')
plt.savefig('P_phishing.jpg')

In [ ]:
dynamic_yfit

In [ ]:
def get_sample_trival_feature(train_data):
    #feature between from and block:
    from_block_feature = train_data.groupby(['from'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('from_block_')
    from_value_feature = train_data.groupby(['from'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('from_value_')
    from_feature = pd.concat([from_block_feature, from_value_feature], axis = 1)
    #from_feature = from_feature.merge(left_on='from', right=train_address, right_on='address', how='right')
    to_block_feature = train_data.groupby(['to'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('to_block_')
    to_value_feature = train_data.groupby(['to'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('to_value_')
    to_feature = pd.concat([to_block_feature, to_value_feature], axis = 1)
    from_feature.reset_index(inplace=True)
    to_feature.reset_index(inplace=True)
    #to_feature = to_feature.merge(left_on='to', right=train_address, right_on='address', how='right')
    trival_feature = pd.merge(left=from_feature, right=to_feature, left_on='from', right_on='to', how='right')
    trival_feature['address'] = trival_feature['to']
    trival_feature.drop(['from', 'to'], axis = 1, inplace=True)
    return trival_feature
def get_sample_feature(train_data):
    trival_feature = get_sample_trival_feature(train_data)
    from_to_degree_feature, to_from_degree_feature = get_network_feature(train_data)
    feature = trival_feature.merge(right=from_to_degree_feature, left_on='address', right_on='from', how='left')
    feature = feature.merge(right=to_from_degree_feature, left_on='address', right_on='to',how='left')
    feature.drop(['from','to'], inplace=True, axis = 1)
    return feature

### phishing 金额对比

In [ ]:
sample_addr_data.iloc[0]['value']

In [ ]:
phishing_sum = [sample_addr_data.iloc[0]['value']]
model_expected_sum = [sample_addr_data.iloc[0]['value']]
for i in range(1, len(sample_addr_data) - 1):
    phishing_sum.append(phishing_sum[-1] + sample_addr_data.iloc[i]['value'])
    model_expected_sum.append(model_expected_sum[-1] + sample_addr_data.iloc[i]['value'] * (1 - sample_dynamic_res[i - 1]))

In [ ]:
pd.Series(phishing_sum).plot(label='Loss without model warning')
pd.Series(model_expected_sum).plot(label='Loss with model warning')
plt.xlabel('Number of transactions')
plt.ylabel('Loss(Eth)')
plt.legend()
plt.savefig('Loss_with_model.jpg')

In [ ]:
1 - model_expected_sum[-1] / phishing_sum[-1]

In [ ]:
tmp_pre = 0.819627554383652

tmp_recall = 0.8049612759757687
(tmp_pre * tmp_recall * 2) / (tmp_pre + tmp_recall)

In [ ]:
all_address_set = set(train_data['from']).union(set(train_data['from']))

In [ ]:
len(all_address_set)

In [ ]:
pd.DataFrame(lgb_model.feature_name()).to_csv('feature_names.csv')

## 不同feature frac 的影响

In [ ]:

feature_frac_precision = []
feature_frac_recall = []
feature_frac_auc = []
for i in range(20):
    X_train, y_train = feature_matrix_to_train_matrix(feature, train_label)
    dtrain = lgb.Dataset(X_train, label = y_train)
    feature_frac_precision_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_pre_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_pre_evalMetric'),
           early_stopping_rounds= 100)
    feature_frac_recall_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_recall_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_recall_evalMetric'),
           early_stopping_rounds= 100)   
    feature_frac_auc_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_auc_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_auc_evalMetric'),
           early_stopping_rounds= 100)      
    feature_frac_precision.append(max(feature_frac_precision_tmp['res-mean']))
    feature_frac_recall.append(max(feature_frac_recall_tmp['res-mean']))
    feature_frac_auc.append(max(feature_frac_auc_tmp['res-mean']))

In [ ]:
np.mean(feature_frac_precision)

In [ ]:
np.mean(feature_frac_recall)

In [ ]:
np.mean(feature_frac_auc)